У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

###### Цільова змінна: Segmentation (класи A, B, C, D).
Ідентифікатор: ID — можна видалити, бо він не несе інформаційної цінності.
Категоріальні змінні: Gender, Ever_Married, Graduated, Profession, Spending_Score, Var_1.
Числові змінні: Age, Work_Experience, Family_Size.

###### Присутні пропущені значення в колонках:
Ever_Married — 140 пропущених.
Graduated — 78 пропущених.
Profession — 124 пропущених.
Work_Experience — 829 пропущених.
Family_Size — 335 пропущених.
Var_1 — 76 пропущених.

###### Далі підемо за таким планом:
Видалимо колонку ID.
Обробимо пропущені значення:
категоріальні — заповнимо модою,
числові — медіаною.
Закодуємо категоріальні ознаки (OneHotEncoder).

Розділимо дані на X та y, потім — на тренувальну та тестову вибірки (80/20).
Збережемо індекси категоріальних колонок для SMOTENC

In [2]:
import pandas as pd

In [3]:
file_path = "C:/Users/linam/ML_Projects/HW/2.2/2.2_Незбалансована багатокласова класифікація/customer_segmentation_train.csv"
df = pd.read_csv(file_path)

In [4]:
# Перевіримо основну інформацію про датасет
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


(None,
        ID  Gender Ever_Married  Age Graduated     Profession  Work_Experience  \
 0  462809    Male           No   22        No     Healthcare              1.0   
 1  462643  Female          Yes   38       Yes       Engineer              NaN   
 2  466315  Female          Yes   67       Yes       Engineer              1.0   
 3  461735    Male          Yes   67       Yes         Lawyer              0.0   
 4  462669  Female          Yes   40       Yes  Entertainment              NaN   
 
   Spending_Score  Family_Size  Var_1 Segmentation  
 0            Low          4.0  Cat_4            D  
 1        Average          3.0  Cat_4            A  
 2            Low          1.0  Cat_6            B  
 3           High          2.0  Cat_6            B  
 4           High          6.0  Cat_6            A  )

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [7]:
# Видаляємо колонку ID
df.drop(columns=["ID"], inplace=True)

In [8]:
# Визначаємо категоріальні та числові колонки
categorical_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
numerical_cols = ['Age', 'Work_Experience', 'Family_Size']

In [9]:
# Імп'ютери
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='median')

In [10]:
# Заповнення пропущених значень
df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])
df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

In [11]:
# Відокремлюємо ознаки та цільову змінну
X = df.drop(columns=["Segmentation"])
y = df["Segmentation"]

In [12]:
# Розділення на train і test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
# Знаходимо індекси категоріальних колонок у повному датасеті
categorical_indices = [X.columns.get_loc(col) for col in categorical_cols]

In [14]:
X_train.shape, X_test.shape, categorical_indices

((6454, 9), (1614, 9), [0, 1, 3, 4, 6, 8])

In [25]:
from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

# Копії, щоб не зіпсувати X_train
X_train_encoded = X_train.copy()

# Закодуємо категоріальні ознаки
ordinal_encoder = OrdinalEncoder()
X_train_encoded[categorical_cols] = ordinal_encoder.fit_transform(X_train_encoded[categorical_cols])

# Індекси категоріальних ознак
categorical_indices = [X_train_encoded.columns.get_loc(col) for col in categorical_cols]

# SMOTENC
smote = SMOTENC(categorical_features=categorical_indices, random_state=42)
X_smote, y_smote = smote.fit_resample(X_train_encoded, y_train)

# SMOTE-Tomek
smote_tomek = SMOTETomek(
    smote=SMOTENC(categorical_features=categorical_indices, random_state=42),
    random_state=42
)
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X_train_encoded, y_train)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [30]:
# Моделі
clf_original = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
clf_smote = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
clf_smote_tomek = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))

# Навчання
clf_original.fit(X_train_encoded, y_train)
clf_smote.fit(X_smote, y_smote)
clf_smote_tomek.fit(X_smote_tomek, y_smote_tomek)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
                                                 random_state=42))

In [31]:
X_test_encoded = X_test.copy()
X_test_encoded[categorical_cols] = ordinal_encoder.transform(X_test_encoded[categorical_cols])

In [32]:
y_pred_original = clf_original.predict(X_test_encoded)
y_pred_smote = clf_smote.predict(X_test_encoded)
y_pred_smote_tomek = clf_smote_tomek.predict(X_test_encoded)

In [33]:
print("Original Data:")
print(classification_report(y_test, y_pred_original))

print("\nSMOTE Data:")
print(classification_report(y_test, y_pred_smote))

print("\nSMOTE-Tomek Data:")
print(classification_report(y_test, y_pred_smote_tomek))

Original Data:
              precision    recall  f1-score   support

           A       0.39      0.39      0.39       394
           B       0.41      0.08      0.14       372
           C       0.47      0.64      0.54       394
           D       0.59      0.79      0.67       454

    accuracy                           0.49      1614
   macro avg       0.46      0.48      0.44      1614
weighted avg       0.47      0.49      0.45      1614


SMOTE Data:
              precision    recall  f1-score   support

           A       0.41      0.40      0.41       394
           B       0.37      0.14      0.20       372
           C       0.48      0.63      0.54       394
           D       0.62      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.47      0.49      0.46      1614
weighted avg       0.48      0.50      0.47      1614


SMOTE-Tomek Data:
              precision    recall  f1-score   support

           A       0.41     

## SMOTENC

In [34]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

# індекси категоріальних ознак
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_cols]

# SMOTENC
smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42)
X_smote, y_smote = smote_nc.fit_resample(X_train_encoded, y_train)

# SMOTE-Tomek
smote_tomek = SMOTETomek(
    smote=SMOTENC(categorical_features=categorical_indices, random_state=42),
    random_state=42
)
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X_train_encoded, y_train)

In [35]:
# Створення моделей
clf_original = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
clf_smote = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
clf_smote_tomek = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))

# Навчання
clf_original.fit(X_train_encoded, y_train)
clf_smote.fit(X_smote, y_smote)
clf_smote_tomek.fit(X_smote_tomek, y_smote_tomek)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
                                                 random_state=42))

In [36]:
# Прогнози
y_pred_original = clf_original.predict(X_test_encoded)
y_pred_smote = clf_smote.predict(X_test_encoded)
y_pred_smote_tomek = clf_smote_tomek.predict(X_test_encoded)

# Звіти
print("=== Original Data ===")
print(classification_report(y_test, y_pred_original))

print("\n=== SMOTE ===")
print(classification_report(y_test, y_pred_smote))

print("\n=== SMOTE-Tomek ===")
print(classification_report(y_test, y_pred_smote_tomek))

=== Original Data ===
              precision    recall  f1-score   support

           A       0.39      0.39      0.39       394
           B       0.41      0.08      0.14       372
           C       0.47      0.64      0.54       394
           D       0.59      0.79      0.67       454

    accuracy                           0.49      1614
   macro avg       0.46      0.48      0.44      1614
weighted avg       0.47      0.49      0.45      1614


=== SMOTE ===
              precision    recall  f1-score   support

           A       0.41      0.40      0.41       394
           B       0.37      0.14      0.20       372
           C       0.48      0.63      0.54       394
           D       0.62      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.47      0.49      0.46      1614
weighted avg       0.48      0.50      0.47      1614


=== SMOTE-Tomek ===
              precision    recall  f1-score   support

           A     

 ##### Яку метрику ви обрали для порівняння моделей?
Я обрав macro F1-score, оскільки це багатокласова задача. 
Macro F1-score однаково зважує всі класи, незалежно від їх частоти.
Це важливо в задачі сегментації клієнтів, де недообслуговування меншого сегмента може бути критичним для бізнесу.

##### 3.1. Як змінилися метрики після застосування SMOTE?

Accuracy майже не змінилася:

було: 0.49 → стало: 0.50 (на 1% вище)

Клас B:

Recall підвищився трохи (з 0.08 → 0.14)

F1 підвищився з 0.14 → 0.20

Клас C:

Recall тримається на тому ж рівні (0.64 → 0.63), precision стабільна

Клас D:

Легке покращення precision/recall/F1

**Загалом: SMOTE трохи підняв якість класифікації для класів B і D, але ефект несильний. Найбільше — по recall для класу B, хоча все ще низький.**

##### 3.2. Який вплив має SMOTE-Tomek порівняно зі звичайним SMOTE?
Клас A:

Recall знизився: 0.40 - 0.36, але precision така ж

Клас B:

Recall трохи впав: 0.14 - 0.17, але precision нижча

Клас C, D:

Результати майже ідентичні до SMOTE

**Висновок: SMOTE-Tomek трохи гірше справляється з класом A, B, ніж SMOTE, але практично не змінює ситуацію з класами C і D.**

##### 3.3. Який метод дав найкращі результати?
Залежить від того, які метрики для тебе найважливіші, але загалом:

Метод	Accuracy	Macro F1	Коментар
Original	- Краще для класу A
SMOTE	- Найкращий загальний баланс
SMOTE-Tomek	- уже схожий до SMOTE, але трохи гірший для класу A

**SMOTE виглядає найкращим варіантом у цьому випадку.**

##### 3.4. Чи покращився класифікатор для менш представлених класів?
Так, зокрема:

Клас B — найменш представлений (372):

Recall: з 0.08 → 0.14 (SMOTE) → 0.17 (SMOTE-Tomek)

F1: з 0.14 → 0.20 → 0.23

**Отже, класифікатор краще "бачить" рідкісні класи, хоча і не ідеально.**

##### 3.5. Чи є сенс використовувати балансування в цій задачі?
Так, сенс є:

Результати для рідкісних класів покращуються

Загальна точність не падає, а навіть трохи зростає

Це покращує справедливість класифікації між класами

Балансування особливо важливе, якщо класи мають різну важливість для бізнесу або менші класи критичні, бо без балансування їх майже не видно (як було з B — recall 0.08).